Nhập data

In [2]:
import os
import fasttext
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy
from num2words import num2words 
from sklearn import metrics

In [3]:
data = open('topic_detection_train.v1.0.txt', newline=None, encoding='utf-8')
# data = open('__label__tai_chinh.txt', newline=None, encoding='utf-8')
data.read()
data.seek(0)
lines = data.readlines()
lowercaseLines = list(map(lambda line: line.lower(), lines))

In [4]:
linesWithoutLinks = list(map(lambda line: re.sub(r"https?\s*:\s*\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", '', line), lowercaseLines))

In [5]:
linesFormattedDot = list(map(lambda line: re.sub(r"\s*\.+\s*(\.*\s*){100}", ' punc ', line), linesWithoutLinks))

In [6]:
linesWithoutWierdChar = list(map(lambda line: re.sub(r'[\W]+', ' ', line), linesFormattedDot))

In [7]:
formattedLines = list(map(lambda line: line.replace('\n', '').split(" ", 1), linesWithoutWierdChar))

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(formattedLines, columns=['label', 'content'])

In [10]:
datalist = df['content'].tolist();
linesWithoutUnderScores = list((map(lambda line: re.sub(r'_+', ' ', line), datalist)))

In [ ]:
def convert_brief_char_to_word(matchobj):
    string = matchobj.group(0)
    foundNum = re.findall('\d+', string);
    result = string
    if len(foundNum) > 0:
        num = foundNum[0]
        left = " " + foundNum[1] + " " if len(foundNum) > 1 else ""
        result = num
        if len(num) < len(string):
            char = string[len(num):len(string)] if left == "" else string[len(num):len(string) - len(left) + 2]
            obj = {
                'đ': 'đồng',
                'k': 'nghìn',
                'tr': 'triệu',
                'h': 'giờ',
                'm': 'mét',
                'm²': 'mét vuông',
                'pn': 'phòng ngủ',
            }
            if obj.get(char) != None:
                result += " " + obj.get(char) + left
            else:
                result += " " + char + left
    return result

In [ ]:
splittedNumAndWordLines = list((map(lambda line: re.sub(r'\d*[a-zđ²]+\d*', convert_brief_char_to_word,line), linesWithoutUnderScores)))

In [ ]:
def convert_num_to_word(matchobj):
    num = matchobj.group(0)
    if len(num) > 7 or num[0] == '0':
        num = " ".join(list(map(lambda char: num2words(int(char), lang='vi'),list(num))))
        return num;
    return num2words(int(num), lang='vi')

In [ ]:
num2wordsLines = list((map(lambda line: re.sub(r'\d+', convert_num_to_word,line), splittedNumAndWordLines)))

In [11]:
# removedOneWordLines = list((map(lambda line: re.sub(r'\s+\w\s+', " ",line), num2wordsLines)))
removedOneWordLines = list((map(lambda line: re.sub(r'\s+\w\s+', " ",line), linesWithoutUnderScores)))

Skip 

In [ ]:
def get_stopwords(documents, threshold=3):
    """
    :param documents: list of documents
    :param threshold:
    :return: list of words has idf <= threshold
    """
    tfidf = TfidfVectorizer(min_df=100)
    tfidf_matrix = tfidf.fit_transform(documents)
    features = tfidf.get_feature_names()
    stopwords = []
    print(min(tfidf.idf_), max(tfidf.idf_), len(features))
    for index, feature in enumerate(features):
        if tfidf.idf_[index] <= threshold:
            stopwords.append(feature)
    return stopwords

In [ ]:
stopwords = get_stopwords(df['content'], threshold=2.5)

In [ ]:
with open('manual_stopwords.txt', 'w', encoding='utf8') as fp:
        for word in stopwords:
            fp.write(word + '\n')

Continue here

In [12]:
lines = open('stopwords.txt', newline=None, encoding='utf-8')
lines.read()
lines.seek(0)
stopwords = lines.readlines()
stopwords = list(map(lambda line: line.replace('\n', ''), stopwords))

In [13]:
stopwords.pop(0)
# stopwords

'\ufeff'

In [14]:
linesWithoutStopwords = list(map(lambda line: " ".join(list(filter(lambda word: stopwords.count(word) == 0, line.split(" ")))), removedOneWordLines))

In [15]:
df['content'] = linesWithoutStopwords

Split data to files

In [ ]:
files = {}
for label, content in zip(list(df['label']), list(df['content'])):
  if files.get(label) == None:
    files[label] = []
  files[label].append(content)

In [ ]:
for label in files.keys():
  store_file(label + '.txt', files[label], len(files[label]) * [label])

In [ ]:
X_train, X_test, y_train, y_test = [[], [], [], []]
for label in files.keys():
    if "__label__tai_chinh, __label__kinh_doanh_va_cong_nghiep".count(label):
        x_tr, x_te, y_tr, y_te = train_test_split(files[label], len(files[label]) * [label], test_size=0.4, random_state=0)        
    else:
        x_tr, x_te, y_tr, y_te = train_test_split(files[label], len(files[label]) * [label], test_size=0.4, random_state=0)
    X_train.extend(x_tr)
    X_test.extend(x_te)
    y_train.extend(y_tr)
    y_test.extend(y_te)

In [ ]:
store_file('created_train.txt', X_train, y_train)
store_file('created_test.txt', X_test, y_test)

Use sklearn k-fold cross validation technique

In [16]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import f1_score

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.4, random_state=0)

In [21]:
store_file('created_train.txt', X_train.tolist(), y_train.tolist())
store_file('created_test.txt', X_test.tolist(), y_test.tolist())

Estimator for fasttext

In [ ]:
class FasttextEstimator(BaseEstimator):

    def __init__(self, model_dir):
        self.model_dir = model_dir
        self.model = None

    def fit(self, features, labels):
        """
        Train fasttext on the given features and labels.
        :param features: a list of documents.
        :param labels: the list of labels associated with the list of features.
        """
        store_file(os.path.join(self.model_dir, "train.txt"), features, labels)
        fasttext.train_supervised(input=os.path.join(self.model_dir, "train.txt"),
                                  epoch=6, lr=1.0, wordNgrams=2, verbose=1, minCount=1, loss='hs', thread=4).save_model('cv_model.bin')
        self.model = fasttext.load_model(os.path.join(self.model_dir, "cv_model.bin"))
        return self

    def score(self, features, labels):
        """
        Compute the macro-f1 score for the predictions on the given features.
        :param features: a list of documents.
        :param labels: the list of labels associated with the list of features.
        :return: f1_score: the macro-f1 score for the predictions on the given features.
        """
        predicted_labels = []
        for feature in features:
            prediction = self.model.predict(feature)
            predicted_label = prediction[0]
            predicted_labels.append(predicted_label)
        return f1_score(labels, predicted_labels, average="macro")

In [20]:
def store_file(output_file, features, labels):
    """Write the training data in fasttext format to disk.
    :param output_file: the name of the output file.
    :param features: the features, a list of strings.
    :param labels: the labels associated with features.
    """
    with open(output_file, 'w', newline='\n', encoding='utf-8') as f:
        for i in range(0, len(features)):
            f.write("%s %s\n" % (labels[i], features[i]))

Fasttext library

In [ ]:
# model = fasttext.train_supervised(input='created_train.txt', autotuneValidationFile='created_test.valid')
# model.save_model('cv_model.bin')

In [ ]:
model = fasttext.load_model("cv_model.bin")
model.test('created_test.txt')

In [ ]:
# model.('mùa thứ nhất')
estimator = FasttextEstimator(model_dir='C:/Users/Dung/Desktop/python/My python note/')
cross_val_score(estimator, df['content'].tolist(), df['label'].tolist(), cv=5)

In [ ]:
# model = fasttext.load_model("cv_model.bin")
# model.get_words()

SGD

In [58]:
text_clf = Pipeline([
     ('vect', CountVectorizer(max_features=5000)),
     ('tfidf', TfidfTransformer(sublinear_tf=True, use_idf=True)),
     ('clf', SGDClassifier(loss='log', penalty='l2')),
])


In [59]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0],
}

In [60]:
gs_clf = GridSearchCV(text_clf, parameters, cv=3, refit = True, iid=False, verbose=2, n_jobs=-1)

In [61]:
gs_clf.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  3.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=5000,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [62]:
gs_clf.best_params_

{'clf__alpha': 0.0001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [63]:
predicted = gs_clf.predict(X_test)

In [64]:
numpy.mean(predicted == y_test)  

0.87015625

In [65]:
print(metrics.classification_report(y_test, predicted))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.96      0.99      0.97       341
         __label__con_nguoi_va_xa_hoi       0.95      0.97      0.96       161
               __label__cong_nghe_moi       0.00      0.00      0.00        13
            __label__do_an_va_do_uong       0.97      0.99      0.98       948
                     __label__du_lich       0.98      0.96      0.97       366
                    __label__giai_tri       0.97      1.00      0.99        74
                    __label__giao_duc       0.92      0.96      0.94       269
                  __label__giao_thong       1.00      0.86      0.93        37
                    __label__khoa_hoc       0.96      0.83      0.89        65
   __label__kinh_doanh_va_cong_nghiep       0.61      0.74      0.67       950
         __label__lam_dep_va_the_hinh       0.99      0.92      0.95       125
 __label__mang_internet_va_vien_thong       0.99   

C:\Users\Dung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
matrix =  metrics.confusion_matrix(y_test, predicted)

Trial

In [66]:
outlier_y_train = []
outlier_x_train = []
outlier_y_test = []
outlier_x_test = []

outliers = ", ".join(['__label__cong_nghe_moi','__label__tai_chinh', '__label__kinh_doanh_va_cong_nghiep'])

for label, content in zip(y_train, X_train):
    if outliers.count(label) == 1:
        outlier_y_train.append(label)
        outlier_x_train.append(content)
for label, content in zip(y_test, X_test):
    if outliers.count(label) == 1:
        outlier_y_test.append(label)
        outlier_x_test.append(content)
temp_y_train = ['__label__outlier' if outliers.count(label) else label
               for label in y_train]
temp_y_test = ['__label__outlier' if outliers.count(label) else label
               for label in y_test]
store_file('outlier_train.txt', outlier_x_train, outlier_y_train)
store_file('outlier_test.txt', outlier_x_test, outlier_y_test)
store_file('train_temp.txt', X_train.tolist(), temp_y_train)
store_file('test_temp.txt', X_test.tolist(), temp_y_test)

In [72]:
text_clf = Pipeline([
     ('vect', CountVectorizer(max_features=5000, ngram_range=(1,2))),
     ('tfidf', TfidfTransformer(sublinear_tf=True, use_idf=True)),
     ('clf', SGDClassifier(loss='log', penalty='l2', alpha=1e-3)),
])

In [73]:
text_clf.fit(X_train, temp_y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=5000, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='log',
                        

In [74]:
predictions_SGD = text_clf.predict(X_test)
print(metrics.classification_report(temp_y_test,predictions_SGD))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.97      0.86      0.91       341
         __label__con_nguoi_va_xa_hoi       0.99      0.86      0.92       161
            __label__do_an_va_do_uong       0.92      0.98      0.95       948
                     __label__du_lich       0.98      0.86      0.91       366
                    __label__giai_tri       1.00      0.92      0.96        74
                    __label__giao_duc       0.94      0.76      0.84       269
                  __label__giao_thong       1.00      0.08      0.15        37
                    __label__khoa_hoc       0.00      0.00      0.00        65
         __label__lam_dep_va_the_hinh       1.00      0.45      0.62       125
 __label__mang_internet_va_vien_thong       1.00      0.90      0.95       233
__label__may_tinh_va_thiet_bi_dien_tu       1.00      0.68      0.81        88
                     __label__mua_sam       0.89   

C:\Users\Dung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Outlier

In [ ]:
Tfidf_vect_outlier = TfidfVectorizer(max_features=1000, ngram_range=(1,2), max_df=0.5, sublinear_tf=True)
Tfidf_vect_outlier.fit(outlier_x_train)

In [48]:
outlier_true_x_test = []
outlier_true_y_test = []
x_list = X_test.tolist()
y_list = y_test.tolist()
# for content in X_test.tolist():
#     if outlier_x_test.count(content) == 1:
#         index = outlier_x_test.index(content)
#         outlier_true_x_test.append(outlier_x_test[index])
#         outlier_true_y_test.append(outlier_y_test[index])
for index, label in enumerate(predictions_SGD):
    if label == '__label__outlier':
        outlier_true_x_test.append(x_list[index])
        outlier_true_y_test.append(y_list[index])   
store_file('outlier_true_test.txt', outlier_true_x_test, outlier_true_y_test)

In [49]:
# Train_X_outlier_Tfidf = Tfidf_vect_outlier.transform(outlier_x_train)
# Test_X_outlier_Tfidf = Tfidf_vect_outlier.transform(outlier_true_x_test)

In [50]:
text_outlier_clf = Pipeline([
     ('vect', CountVectorizer(ngram_range=(1,1))),
     ('tfidf', TfidfTransformer(sublinear_tf=True, use_idf=True)),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42, tol=None)),
])

In [51]:
text_outlier_clf.fit(outlier_x_train, outlier_y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [52]:
predictions_SGD_outlier = text_outlier_clf.predict(outlier_true_x_test)
print(metrics.classification_report(outlier_true_y_test,predictions_SGD_outlier))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.00      0.00      0.00         4
         __label__con_nguoi_va_xa_hoi       0.00      0.00      0.00         1
               __label__cong_nghe_moi       0.00      0.00      0.00         8
            __label__do_an_va_do_uong       0.00      0.00      0.00         1
                     __label__du_lich       0.00      0.00      0.00         5
                    __label__giao_duc       0.00      0.00      0.00        18
                  __label__giao_thong       0.00      0.00      0.00         3
                    __label__khoa_hoc       0.00      0.00      0.00         5
   __label__kinh_doanh_va_cong_nghiep       0.59      0.89      0.71       916
         __label__lam_dep_va_the_hinh       0.00      0.00      0.00         1
 __label__mang_internet_va_vien_thong       0.00      0.00      0.00         5
__label__may_tinh_va_thiet_bi_dien_tu       0.00   

C:\Users\Dung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# sorted(Tfidf_vect_outlier.vocabulary_, reverse=True)
Tfidf_vect_outlier.vocabulary_

In [55]:
# predictions_SGD_outlier = predictions_SGD_outlier.tolist()
predictions = []
for label in predictions_SGD:
    if label == '__label__outlier':
        predictions.append(predictions_SGD_outlier.pop(0))
    else:
        predictions.append(label)

In [57]:
print(metrics.classification_report(y_test,predictions))

                                       precision    recall  f1-score   support

                   __label__chinh_tri       0.95      0.98      0.96       341
         __label__con_nguoi_va_xa_hoi       0.93      0.98      0.95       161
               __label__cong_nghe_moi       1.00      0.15      0.27        13
            __label__do_an_va_do_uong       0.96      1.00      0.98       948
                     __label__du_lich       0.96      0.94      0.95       366
                    __label__giai_tri       0.96      1.00      0.98        74
                    __label__giao_duc       0.92      0.91      0.92       269
                  __label__giao_thong       1.00      0.68      0.81        37
                    __label__khoa_hoc       0.95      0.63      0.76        65
   __label__kinh_doanh_va_cong_nghiep       0.59      0.86      0.70       950
         __label__lam_dep_va_the_hinh       1.00      0.92      0.96       125
 __label__mang_internet_va_vien_thong       0.97   

In [ ]:
fasttext.train_supervised(input='outlier_train.txt', maxn=0, minn=0, bucket=785249, neg=5, ws=5, dim=53,
                                  epoch=2, lr=1.0, wordNgrams=5, verbose=2, minCount=1, loss='softmax', thread=4).save_model('cv_model_outlier.bin')

In [ ]:
# model = fasttext.load_model("cv_model_outlier.bin")
model.test_label('outlier_true_test.txt')